In [5]:
import os
import numpy as np
from sklearn import mixture
import pandas as pd
import re
import scipy.stats as stats
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

In [6]:
def ReadListOfTextFilesFromDirectoryWalk(path):
    """Read all text files from given directory and its sudirectories and return list of text files path"""
    files = []
    # r=root, d=directories, f = files
    for r, d, f in os.walk(path):
        for file in f:
            if '.txt' in file:
                files.append(os.path.join(r, file))
    return files


In [7]:
def CreateListOfTuplesFromFile(files):
#Reading file and create tuple 
    items = []
    for file in files:
        filename=file.split('\\')[-1].split('.')[0]
        f=open(file, "r")
        for line in f:
            words= line.rstrip('\n').split(sep=" ")
            items.append((words[0],words[1],words[2],filename))
    return items

In [8]:
#Creating dataframes
def CreateDataFrame(items,columns):
    """Taking list of tuples and return dataframe"""
    df = pd.DataFrame.from_records(items, columns=['Key', 'EventType','Time','User'])
    return df

In [9]:
def ParseAlphabetsKeys(df):
    """Return only alphabets records"""
    dfAlphabets=df[df["Key"].str.match('^.*[A-Z]$')]
    return dfAlphabets

In [10]:
def GetTimeDifferenceofKeyDownDown(data):
    prev=0
    rows=[]
    for index,row in data.iterrows():
        if prev==0 and row[1]=="KeyDown":
            prevRow=row
            prev=1
        elif row[1]=="KeyDown":
            rows.append([ prevRow[0], row[0], int(row[2])- int(prevRow[2]),row[3]])
            prevRow=row
    return rows

In [11]:
def GetDictionaryOfFeatureVectors(keyDownDownTimeDifference):
    """
    Take List of arrays and return dictionary of 26*26 vector for each user
    """
    dictFV=dict()
    for xi in keyDownDownTimeDifference:
        if xi[3] not in dictFV:
            dictFV[xi[3]]=np.zeros(26*26,dtype=object)
            
        index=(ord(xi[0])-65)*26+np.absolute(ord(xi[1])-65)
        if dictFV[xi[3]][index]==0:
            dictFV[xi[3]][index]=[]
        if xi[2] < 1000:
            dictFV[xi[3]][index].append(xi[2])
    return dictFV

In [12]:
def GetGMMFormatDataFromDataFrame(usersFV):
    GMMData=[]
    for user in usersFV.columns:
        maxLength=0
        for userF in usersFV[user]:
            if userF==0:
                userF=[]
            featureLenght=len(userF)
            if featureLenght > maxLength:
                maxLength=featureLenght
        userDF=pd.DataFrame(index=range(0,676))
        userData=np.zeros((maxLength,676))
        row=0
        for userF in usersFV[user]:
            column=0
            if userF==0:
                userF=[]
            availableValuesCount=len(userF)
            sum=0
            for value in userF:
                userData[column][row]=value
                column=column+1
                sum=sum+value
            if availableValuesCount==0:
                mean=0
            else:
                mean=sum/availableValuesCount
            gussianValues=np.random.normal(mean,3,maxLength-availableValuesCount)
            for value in gussianValues:
            
                userData[column][row]=value
                column=column+1
        
            row=row+1
        print(userData.shape)
        if len(GMMData)==0:
            GMMData=userData
        else:
            GMMData=np.append(GMMData,userData,axis=0)
    return GMMData


In [13]:
def GetAverageFeatureVectors(FeatureVectors):
    
    averageFV= np.zeros(26*26,dtype=object)
    index=0
    for xi in FeatureVectors:
        tempSum=np.array(xi).sum()
        if type(xi)== list:
            averageFV[index]=tempSum/len(xi)
        index=index+1
    return averageFV

In [14]:
def TrainModel(path):
    """Get path of data files and train a GMM model"""
    files=ReadListOfTextFilesFromDirectoryWalk(path)
    # I am training on first 10 users
    dataFileFormat=CreateListOfTuplesFromFile(files[0:6:2])
    #print(dataFileFormat[0])
    df=CreateDataFrame(dataFileFormat,columns=['Key', 'EventType','Time','User'])
    alphabetsDF=ParseAlphabetsKeys(df)
    keyDownDownTimeDifference=GetTimeDifferenceofKeyDownDown(data=alphabetsDF)
    FeatureVectors=GetDictionaryOfFeatureVectors(keyDownDownTimeDifference) 
    usersFV=pd.DataFrame.from_dict(FeatureVectors)
    data= GetGMMFormatDataFromDataFrame(usersFV)
    g = mixture.GMM(n_components=3)
    print(len(data))
    g.fit(data)
    return g
    

In [15]:
def TestModel(path,g):
    """Get path of data files and test a GMM model"""
    GMMData=[]
    files=ReadListOfTextFilesFromDirectoryWalk(path)
    dataFileFormat=CreateListOfTuplesFromFile(files[0:6:2])
    df=CreateDataFrame(dataFileFormat,columns=['Key', 'EventType','Time','User'])
    
    alphabetsDF=ParseAlphabetsKeys(df)
    keyDownDownTimeDifference=GetTimeDifferenceofKeyDownDown(data=alphabetsDF)
    FeatureVectors=GetDictionaryOfFeatureVectors(keyDownDownTimeDifference) 
    usersFV=pd.DataFrame.from_dict(FeatureVectors)
    for user in FeatureVectors:
        print(user)
        data=GetAverageFeatureVectors(FeatureVectors[user])
        dataNP=np.array(data,dtype=np.float64)
        dataNP[np.isnan(dataNP)]=0
        dataNP=dataNP.reshape((1,676))
        
        
        if len(GMMData)==0:
            GMMData=dataNP
        else:
            GMMData=np.append(GMMData,dataNP,axis=0)
    print(GMMData.shape)
    print(len(GMMData))
    
    print(g.predict_proba(GMMData))
    return g.predict(GMMData)

In [70]:
GMM= TrainModel("..\\UB_keystroke_dataset\\S0")
GMM.weights_

(65, 676)
(65, 676)
(65, 676)
195


C:\Users\Muhammad Umsan\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class GMM is deprecated; The class GMM is deprecated in 0.18 and will be  removed in 0.20. Use class GaussianMixture instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\Muhammad Umsan\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:77: DeprecationWarning: Function distribute_covar_matrix_to_match_covariance_type is deprecated; The function distribute_covar_matrix_to_match_covariance_typeis deprecated in 0.18 and will be removed in 0.20.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\Muhammad Umsan\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:77: DeprecationWarning: Function log_multivariate_normal_density is deprecated; The function log_multivariate_normal_density is deprecated in 0.18 and will be removed in 0.20.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\Muhammad Umsan\Anaconda3\lib\site-packages\sklearn\utils\deprec

array([0.33333333, 0.33333333, 0.33333333])

In [73]:
predictions=TestModel("..\\UB_keystroke_dataset\\S1",GMM)

001100
002100
003100
(3, 676)
3
[[0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]]


C:\Users\Muhammad Umsan\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in double_scalars
  
C:\Users\Muhammad Umsan\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:77: DeprecationWarning: Function log_multivariate_normal_density is deprecated; The function log_multivariate_normal_density is deprecated in 0.18 and will be removed in 0.20.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\Muhammad Umsan\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:77: DeprecationWarning: Function log_multivariate_normal_density is deprecated; The function log_multivariate_normal_density is deprecated in 0.18 and will be removed in 0.20.
  warnings.warn(msg, category=DeprecationWarning)


In [74]:
predictions

array([1, 2, 1], dtype=int64)

In [83]:
def GetKDEFormatDataFromDataFrame(usersFV):
    GMMData=[]
    TargetUser=[]
    for user in usersFV.columns:
        print(user)
        maxLength=0
        for userF in usersFV[user]:
            if userF==0:
                userF=[]
            featureLenght=len(userF)
            if featureLenght > maxLength:
                maxLength=featureLenght
        userDF=pd.DataFrame(index=range(0,676))
        userData=np.zeros((maxLength,676))
        a=np.empty(maxLength)
        a.fill(user)
        if len(TargetUser) < 1:
            TargetUser=a
        else:
            TargetUser=np.concatenate((TargetUser,a))
        row=0
        for userF in usersFV[user]:
            column=0
            if userF==0:
                userF=[]
            availableValuesCount=len(userF)
            sum=0
            for value in userF:
                userData[column][row]=value
                column=column+1
                sum=sum+value
            if availableValuesCount==0:
                mean=0
            else:
                mean=sum/availableValuesCount
            gussianValues=np.random.normal(mean,3,maxLength-availableValuesCount)
            for value in gussianValues:
            
                userData[column][row]=value
                column=column+1
        
            row=row+1
        print(userData.shape)
        if len(GMMData)==0:
            GMMData=userData
        else:
            GMMData=np.append(GMMData,userData,axis=0)
            
        
    return GMMData,TargetUser

In [91]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.neighbors import KernelDensity
from statsmodels.nonparametric.kernel_density import KDEMultivariate

class KDEClassifier(BaseEstimator, ClassifierMixin):
    """Bayesian generative classification based on KDE
    
    Parameters
    ----------
    bandwidth : float
        the kernel bandwidth within each class
    kernel : str
        the kernel name, passed to KernelDensity
    """
    def __init__(self, bandwidth=1.0, kernel='gaussian'):
        self.bandwidth = bandwidth
        self.kernel = kernel
        
    def fit(self, X, y):
        self.classes_ = np.sort(np.unique(y))
        training_sets = [X[y == yi] for yi in self.classes_]
        self.models_ = [KernelDensity(bandwidth=self.bandwidth,
                                      kernel=self.kernel).fit(Xi)
                        for Xi in training_sets]
        self.logpriors_ = [np.log(Xi.shape[0] / X.shape[0])
                           for Xi in training_sets]
        return self
        
    def predict_proba(self, X):
        logprobs = np.array([model.score_samples(X)
                             for model in self.models_]).T
        result = np.exp(logprobs + self.logpriors_)
        return result / result.sum(1, keepdims=True)
        
    def predict(self, X):
        return self.classes_[np.argmax(self.predict_proba(X), 1)]

In [92]:
files=ReadListOfTextFilesFromDirectoryWalk("..\\UB_keystroke_dataset\\S1")
# I am training on first 10 users
dataFileFormat=CreateListOfTuplesFromFile(files[0:6:2])
#print(dataFileFormat[0])
df=CreateDataFrame(dataFileFormat,columns=['Key', 'EventType','Time','User'])
alphabetsDF=ParseAlphabetsKeys(df)
keyDownDownTimeDifference=GetTimeDifferenceofKeyDownDown(data=alphabetsDF)
FeatureVectors=GetDictionaryOfFeatureVectors(keyDownDownTimeDifference) 
usersFV=pd.DataFrame.from_dict(FeatureVectors)
X,y=GetKDEFormatDataFromDataFrame(usersFV)
KDE=KDEClassifier()
KDE.fit(X,y)
KDE.predict_proba(X)


001100
(62, 676)
002100
(57, 676)
003100
(57, 676)


array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0

In [93]:
files=ReadListOfTextFilesFromDirectoryWalk("..\\UB_keystroke_dataset\\S1")
# I am training on first 10 users
dataFileFormat=CreateListOfTuplesFromFile(files[0:6:2])
#print(dataFileFormat[0])
df=CreateDataFrame(dataFileFormat,columns=['Key', 'EventType','Time','User'])
alphabetsDF=ParseAlphabetsKeys(df)
keyDownDownTimeDifference=GetTimeDifferenceofKeyDownDown(data=alphabetsDF)
FeatureVectors=GetDictionaryOfFeatureVectors(keyDownDownTimeDifference) 
usersFV=pd.DataFrame.from_dict(FeatureVectors)
X_test,y=GetKDEFormatDataFromDataFrame(usersFV)
KDE.predict(X_test)


001100
(62, 676)
002100
(57, 676)
003100
(57, 676)


C:\Users\Muhammad Umsan\Anaconda3\lib\site-packages\ipykernel_launcher.py:33: RuntimeWarning: invalid value encountered in true_divide


array([1100., 1100., 1100., 1100., 1100., 1100., 1100., 1100., 1100.,
       1100., 1100., 1100., 1100., 1100., 1100., 1100., 1100., 1100.,
       1100., 1100., 1100., 1100., 1100., 1100., 1100., 1100., 1100.,
       1100., 1100., 1100., 1100., 1100., 1100., 1100., 1100., 1100.,
       1100., 1100., 1100., 1100., 1100., 1100., 1100., 1100., 1100.,
       1100., 1100., 1100., 1100., 1100., 1100., 1100., 1100., 1100.,
       1100., 1100., 1100., 1100., 1100., 1100., 1100., 1100., 1100.,
       1100., 1100., 1100., 1100., 1100., 1100., 1100., 1100., 1100.,
       1100., 1100., 1100., 1100., 1100., 1100., 1100., 1100., 1100.,
       1100., 1100., 1100., 1100., 1100., 1100., 1100., 1100., 1100.,
       1100., 1100., 1100., 1100., 1100., 1100., 1100., 1100., 1100.,
       1100., 1100., 1100., 1100., 1100., 1100., 1100., 1100., 1100.,
       1100., 1100., 1100., 1100., 1100., 1100., 1100., 1100., 1100.,
       1100., 1100., 1100., 1100., 1100., 1100., 1100., 1100., 1100.,
       1100., 1100.,

In [ ]:
files=ReadListOfTextFilesFromDirectoryWalk(path)
dataFileFormat=CreateListOfTuplesFromFile(files[0:6:2])
df=CreateDataFrame(dataFileFormat,columns=['Key', 'EventType','Time','User'])

alphabetsDF=ParseAlphabetsKeys(df)
keyDownDownTimeDifference=GetTimeDifferenceofKeyDownDown(data=alphabetsDF)
FeatureVectors=GetDictionaryOfFeatureVectors(keyDownDownTimeDifference) 
usersFV=pd.DataFrame.from_dict(FeatureVectors)
for user in FeatureVectors:
    print(user)
    data=GetAverageFeatureVectors(FeatureVectors[user])
    dataNP=np.array(data,dtype=np.float64)
    dataNP[np.isnan(dataNP)]=0
    dataNP=dataNP.reshape((1,676))


    if len(GMMData)==0:
        GMMData=dataNP
    else:
        GMMData=np.append(GMMData,dataNP,axis=0)
print(GMMData.shape)